<a href="https://www.kaggle.com/code/volt3000/fine-tune-llama-3-instruct-8b-on-codesearchnet-alp?scriptVersionId=190453583" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Fine-tune Llama-3-8B-Instruct with Unsloth on CodeSearchNet

> Note: This notebooks runs best when it's accelerated with Nvidia T4(s) or GPU(s) of similar architecture 

## Download, Install and Import Dependencies

In [1]:
%%time
!mamba install --force-reinstall aiohttp -y
!pip install -U "xformers<0.0.26" --index-url https://download.pytorch.org/whl/cu121
!pip install "unsloth[kaggle-new] @ git+https://github.com/unslothai/unsloth.git"

# Temporary fix for https://github.com/huggingface/datasets/issues/6753
!pip install datasets==2.16.0 fsspec==2023.10.0 gcsfs==2023.10.0

import os
os.environ["WANDB_DISABLED"] = "true"


Looking for: ['aiohttp']

warning  libmamba Cache file "/opt/conda/pkgs/cache/c6f2354e.json" was modified by another program
[+] 0.0s
[+] 0.1s
rapidsai/linux-64 (check zst) ━━━━━━━━━━━━━━━   0.0 B @  ??.?MB/s Checking  0.1s[+] 0.2s
rapidsai/linux-64 (check zst) ━━━━━━━━━━━━━━━   0.0 B @  ??.?MB/s Checking  0.2s[+] 0.3s
rapidsai/linux-64 (check zst) ━━━━━━━━━━━━━━━   0.0 B @  ??.?MB/s Checking  0.3s[+] 0.4s
rapidsai/linux-64 (check zst) ━━━━━━━━━━━━━━━   0.0 B @  ??.?MB/s Checking  0.4s[+] 0.5s
rapidsai/linux-64 (check zst) ━━━━━━━━━━━━━━━   0.0 B @  ??.?MB/s Checking  0.5s[+] 0.6s
rapidsai/linux-64 (check zst) ━━━━━━━━━━━━━━━   0.0 B @  ??.?MB/s Checking  0.6s[+] 0.7s
rapidsai/linux-64 (check zst) ━━━━━━━━━━━━━━━   0.0 B @  ??.?MB/s Checking  0.7s[+] 0.8s
rapidsai/linux-64 (check zst) ━━━━━━━━━━━━━━━   0.0 B @  ??.?MB/s Checking  0.8s[+] 0.9s
rapidsai/linux-64 (check zst) ━━━━━━━━━━━━━━━   0.0 B @  ??.?MB/s Checking  0.9srapidsai/linux-64 (check zst)                       Checked  1.0

In [2]:
from unsloth import FastLanguageModel
from trl import SFTTrainer
from transformers import TrainingArguments
import pprint as pp
from datasets import load_dataset
import torch

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


2024-07-30 12:09:09.685445: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-30 12:09:09.685543: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-30 12:09:09.867043: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


## Setup Model and Tokenizer from Unsloth

In [3]:
max_seq_length = 1024
dtype = None
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit
)

==((====))==  Unsloth 2024.8: Fast Llama patching. Transformers = 4.43.3.
   \\   /|    GPU: Tesla T4. Max memory: 14.741 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.2.2+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.25.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/459 [00:00<?, ?B/s]

In [4]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = True, # Set to True if out of memory (default is "unsloth")
    random_state = 8402,
    use_rslora = False,
    loftq_config = None,
)

Unsloth 2024.8 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


## Format CodeSearchNet to Alpaca-styled Format

In [5]:
# alpacaFormatString = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

# ### Instruction:
# {}

# ### Input:
# {}

# ### Response:
# {}"""

# EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN (<|eot_id|>)

# # Define the formatting function to initially drop all the unnecessary columns and rename what we need
# def formatFunctionSample(sample):
#     language = sample['language']
#     instruction = f"Briefly explain what this {language} function does, in the format of a docstring?"
#     inputText = sample['func_code_string']
#     outputText = sample['func_documentation_string']

#     # Returning a dictionary of the necessary columns
#     return {
#         "instruction": instruction,
#         "input": inputText,
#         "output": outputText
#     }

# # Define the function to create the new 'text' column
# def createAlpacaFormatString(sample):
#     instruction = sample['instruction']
#     inputText = sample['input']
#     outputText = sample['output']
    
#     text = alpacaFormatString.format(instruction, inputText, outputText) + EOS_TOKEN
#     sample['text'] = text
    
#     return sample

In [6]:
def formatAndCreateAlpacaString(sample):
    alpacaFormatString = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""
    
    EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN (<|eot_id|>)

    # Extract necessary columns and format them
    language = sample['language']
    instruction = f"Briefly explain what this {language} function does, in the format of a docstring?"
    inputText = sample['func_code_string']
    outputText = sample['func_documentation_string']

    # Create the text column in the Alpaca format
    text = alpacaFormatString.format(instruction, inputText, outputText) + EOS_TOKEN
    
    # Returning a dictionary of the necessary columns including the new 'text' column
    return {
        "instruction": instruction,
        "input": inputText,
        "output": outputText,
        "text": text
    }

In [7]:
dataset = load_dataset("claudios/code_search_net", "python", split="train[:10000]")

# Mapping the existing dataset to the new format keeping only the keys of the dictionary we returned
dataset = dataset.map(formatAndCreateAlpacaString, remove_columns=dataset.column_names)

# Adding the text column to the new dataset
# dataset = dataset.map(createAlpacaFormatString)

Generating train split:   0%|          | 0/412178 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/22176 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/23107 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [8]:
dataset

Dataset({
    features: ['instruction', 'input', 'output', 'text'],
    num_rows: 10000
})

In [9]:
pp.pp(dataset[0])

{'instruction': 'Briefly explain what this python function does, in the format '
                'of a docstring?',
 'input': 'def addidsuffix(self, idsuffix, recursive = True):\n'
          '        """Appends a suffix to this element\'s ID, and optionally '
          'to all child IDs as well. There is sually no need to call this '
          'directly, invoked implicitly by :meth:`copy`"""\n'
          '        if self.id: self.id += idsuffix\n'
          '        if recursive:\n'
          '            for e in self:\n'
          '                try:\n'
          '                    e.addidsuffix(idsuffix, recursive)\n'
          '                except Exception:\n'
          '                    pass',
 'output': "Appends a suffix to this element's ID, and optionally to all child "
           'IDs as well. There is sually no need to call this directly, '
           'invoked implicitly by :meth:`copy`',
 'text': 'Below is an instruction that describes a task, paired with an input

In [10]:
print(dataset[0]['text'])

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Briefly explain what this python function does, in the format of a docstring?

### Input:
def addidsuffix(self, idsuffix, recursive = True):
        """Appends a suffix to this element's ID, and optionally to all child IDs as well. There is sually no need to call this directly, invoked implicitly by :meth:`copy`"""
        if self.id: self.id += idsuffix
        if recursive:
            for e in self:
                try:
                    e.addidsuffix(idsuffix, recursive)
                except Exception:
                    pass

### Response:
Appends a suffix to this element's ID, and optionally to all child IDs as well. There is sually no need to call this directly, invoked implicitly by :meth:`copy`<|end_of_text|>


## Train-test Split

In [18]:
datasetDictionary = dataset.train_test_split(test_size=0.003)

In [19]:
datasetDictionary

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output', 'text'],
        num_rows: 9970
    })
    test: Dataset({
        features: ['instruction', 'input', 'output', 'text'],
        num_rows: 30
    })
})

## Initialize Trainer with Training Arguments

In [20]:
# Test trainer config with the evaluation loop to calculate validation loss
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = datasetDictionary["train"],
    eval_dataset = datasetDictionary["test"],
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = True, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        per_device_eval_batch_size = 2,
        gradient_accumulation_steps = 4,
        fp16_full_eval = True,
        eval_accumulation_steps = 4,
        evaluation_strategy = "steps",
        eval_steps = 1,
        warmup_ratio = 0.1,
        max_steps = 60,
        # num_train_epochs = 1,
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 8402,
        output_dir = "outputs",
        report_to = "none",
    ),
)

/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs
/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:342: UserWarning: You passed `packing=True` to the SFTTrainer, and you are training your model with `max_steps` strategy. The dataset will be iterated until the `max_steps` are reached.
  warnings.warn(


In [21]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.741 GB.
9.162 GB of memory reserved.


In [22]:
trainer.train_dataset

Dataset({
    features: ['input_ids', 'labels'],
    num_rows: 3435
})

In [23]:
trainer.eval_dataset

Dataset({
    features: ['input_ids', 'labels'],
    num_rows: 13
})

## Fine-tune Training Loop

In [24]:
trainerStats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 3,435 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss,Validation Loss
1,1.403300,1.596246
2,1.387900,1.585247
3,1.525600,1.560802
4,1.672800,1.534532
5,1.302600,1.522616
6,1.344900,1.497102
7,1.241100,1.460717
8,1.206700,1.426033
9,1.256400,1.403524
10,1.185800,1.384905


In [25]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory          / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainerStats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainerStats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

2236.4305 seconds used for training.
37.27 minutes used for training.
Peak reserved memory = 9.514 GB.
Peak reserved memory for training = 0.352 GB.
Peak reserved memory % of max memory = 64.541 %.
Peak reserved memory for training % of max memory = 2.388 %.


## Run Inference on Fine-tuned Model

In [26]:
from transformers import TextStreamer

textStreamer = TextStreamer(tokenizer)

In [28]:
alpacaFormatString = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""
    

In [31]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

testFunction = """
def train_custom_transformer(input_dim, model_dim, num_heads, num_layers, output_dim, batch_size, num_epochs, learning_rate):
    class TransformerModel(nn.Module):
        def __init__(self, input_dim, model_dim, num_heads, num_layers, output_dim):
            super(TransformerModel, self).__init__()
            self.embedding = nn.Embedding(input_dim, model_dim)
            self.transformer = nn.Transformer(
                d_model=model_dim, 
                nhead=num_heads, 
                num_encoder_layers=num_layers, 
                num_decoder_layers=num_layers
            )
            self.fc_out = nn.Linear(model_dim, output_dim)
        
        def forward(self, src, tgt):
            src_embedding = self.embedding(src)
            tgt_embedding = self.embedding(tgt)
            transformer_output = self.transformer(src_embedding, tgt_embedding)
            output = self.fc_out(transformer_output)
            return output

    # Create some dummy data
    src_data = torch.randint(0, input_dim, (1000, 10))  # 1000 samples, sequence length of 10
    tgt_data = torch.randint(0, input_dim, (1000, 10))  # 1000 samples, sequence length of 10
    labels = torch.randint(0, output_dim, (1000, 10))   # 1000 samples, sequence length of 10

    dataset = TensorDataset(src_data, tgt_data, labels)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

    # Initialize the model, loss function, and optimizer
    model = TransformerModel(input_dim, model_dim, num_heads, num_layers, output_dim)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # Training loop
    model.train()
    for epoch in range(num_epochs):
        epoch_loss = 0
        for src, tgt, lbl in dataloader:
            optimizer.zero_grad()
            output = model(src, tgt)
            loss = criterion(output.view(-1, output_dim), lbl.view(-1))
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
        
        avg_loss = epoch_loss / len(dataloader)
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {avg_loss:.4f}")

"""

inputs = tokenizer(
[
    alpacaFormatString.format(
        "Briefly explain what this Python function does, in the format of a docstring?", # instruction
        testFunction, # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

_ = model.generate(input_ids = inputs.input_ids, attention_mask = inputs.attention_mask,
                   streamer = textStreamer, max_new_tokens = 128, pad_token_id = tokenizer.eos_token_id)

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Briefly explain what this Python function does, in the format of a docstring?

### Input:

def train_custom_transformer(input_dim, model_dim, num_heads, num_layers, output_dim, batch_size, num_epochs, learning_rate):
    class TransformerModel(nn.Module):
        def __init__(self, input_dim, model_dim, num_heads, num_layers, output_dim):
            super(TransformerModel, self).__init__()
            self.embedding = nn.Embedding(input_dim, model_dim)
            self.transformer = nn.Transformer(
                d_model=model_dim, 
                nhead=num_heads, 
                num_encoder_layers=num_layers, 
                num_decoder_layers=num_layers
            )
            self.fc_out = nn.Linear(model_dim, output_dim)
        
        def forward(self, src, tgt):
            src_emb

In [33]:
from huggingface_hub import notebook_login as hfl

In [35]:
hfl()

In [37]:
# model.push_to_hub_gguf("0xVolt/Llama-3.1-8B-Instruct-SFT-CodeSearchNet", tokenizer)
model.push_to_hub_gguf("0xVolt/Llama-3.1-8B-Instruct-SFT-CodeSearchNet", tokenizer, quantization_method = "f16")

  0%|          | 0/32 [00:00<?, ?it/s]


RuntimeError: [enforce fail at inline_container.cc:595] . unexpected pos 576 vs 470